In [15]:
import cv2
import os
import numpy as np

cap= cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error al abrir el video")
    exit()
#declaracion de variables para fuentes y otros   
font_face = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 0.7
font_scale_info = 0.8
font_color = (0,255,0)
font_color_info = (255,0,255)
font_thickness_info = 2
font_thickness = 2
m=False
key=0
contador=0
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) 
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
#se crea el objeto para la deteccion y decodicion de qr
qrDecoder = cv2.QRCodeDetector()
qrDetected=False
#La crea la ventana principal
cv2.namedWindow('QR')

while(cap.isOpened()):
    #se obtiene el frame de la camara y se analiza para multiples QR
    ret, frame = cap.read()
    qrDetected, decoded_info, points, straight_qrcode =qrDecoder.detectAndDecodeMulti(frame)   
    
    if not ret:
        break
    
    if qrDetected:
        #si se han detectado QR se recorre la lista de elementos detectados
        for s, p in zip(decoded_info, points):
                if s:                    
                    color = (0, 255, 0)
                    frame = cv2.polylines(frame, [p.astype(int)], True, color, 2)
                    cv2.putText(frame,str(s), np.array(p[0].astype(int)), 
                    font_face, font_scale_info, font_color_info, font_thickness_info,
                    cv2.LINE_AA)
                    
                else:
                    color = (0, 0, 255)
                    frame = cv2.polylines(frame, [p.astype(int)], True, color, 2)
                
    else:
        text2=" No se detecto ningun codigo QR"
        cv2.putText(frame,text2, (40, 60), 
                font_face, 1, (0, 0, 255), 1,
                cv2.LINE_AA)

    text = "q = salir, s = guardar, m = mascara"
    cv2.putText(frame,text, (40, 20), 
                font_face, font_scale, font_color, font_thickness,
                cv2.LINE_AA)
    
    if key == ord('m') :
        m= not m
        print(m)        
        
    if m:
        if qrDetected:
            #se crea una mascara de 0 
            mascara= np.zeros((int(height),int(width)),dtype=np.uint8)
            for s, p in zip(decoded_info, points):
                    if s:                        
                        cv2.fillPoly(mascara, [p.astype(int)], (255))                        
                        frame = cv2.bitwise_and(frame, frame, mask = mascara)
                        cv2.putText(frame,str(s), np.array(p[0].astype(int)), 
                                     font_face, font_scale_info, font_color_info, font_thickness_info,
                                    cv2.LINE_AA)
                                                                    
                    else:
                        cv2.fillPoly(mascara, [np.array(p.astype(int))], (255)) 
                        frame = cv2.bitwise_and(frame, frame, mask = mascara)
                        print(mascara) 
                        cv2.putText(frame,str(s), np.array(p[0].astype(int)), 
                                     font_face, font_scale_info, font_color_info, font_thickness_info,
                                    cv2.LINE_AA)                      
                        
    cv2.imshow('QR', frame)    
    key = cv2.waitKey(15)
    
    if key == ord('q'):
        break
    
    if key == ord('s'):        
        if os.path.isfile("./output/captura"+str(contador)+".png"):
            while(os.path.isfile("./output/captura"+str(contador)+".png")):
                contador+=1
        cv2.imwrite(("./output/captura"+str(contador)+".png"), frame)       
 
cap.release()
cv2.destroyAllWindows()